### 4.4.3 滑动平均
衰减率：`decay = min{decay, (1+num_updates)/(10+num_updates)}`

影子变量：`shadow_variable = decay*shadow_variable + (1-decay)*variable`

滑动平均不会改变变量本身的取值，而是会维护一个影子变量来记录其滑动平均值。

In [2]:
import tensorflow as tf

#### 1. 定义变量及滑动平均类

In [3]:
# 定义变量用于计算滑动平均，初始值为0
v1 = tf.Variable(0, dtype=tf.float32)
# step变量模拟神经网络中迭代的轮数，可用于动态控制衰减率
step = tf.Variable(0, trainable=False)

# 定义一个滑动平均类（class）
# 初始化衰减率（0.99），控制衰减率的变量（step）
ema = tf.train.ExponentialMovingAverage(decay=0.99, num_updates=step)
# 每一次操作的时候，列表变量[v1]都会被更新
maintain_averages_op = ema.apply([v1]) 

#### 2. 查看不同迭代中变量取值的变化。tf.assign用于变更变量的数值

In [4]:
with tf.Session() as sess:
    
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # 通过ema.average(v1)获取滑动平均之后变量的取值。初始化之后二者都为0
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新变量v1的值为5
    sess.run(tf.assign(v1, 5))
    # 更新v1的滑动平均值
    # 衰减率：min{0.99, (1+step)/(10+step)=0.1} = 0.1, step=0
    # v1的滑动平均：0.1*0 + (1-0.1)*5 = 4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新step和v1的取值
    sess.run(tf.assign(step, 10000))  
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新一次v1的滑动平均值
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))


[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
